In [39]:
import csv
import numpy
import sys
import wfdb
from wfdb import processing

ann_str_to_num  = {
    '(AFIB\x00': 0,
    '(ASYS\x00': 1,
    '(B\x00': 2,
    '(BI\x00': 3,
    '(HGEA\x00': 4,
    '(N\x00': 5,
    '(NSR\x00': 5,
    '(NOD\x00': 6,
    '(NOISE\x00': 7,
    '(PM\x00': 8,
    '(SBR\x00': 9,
    '(SVTA\x00': 10,
    '(VER\x00': 11,
    '(VF\x00': 12,
    '(VFIB\x00': 12,
    '(VFL\x00': 13,
    '(VT\x00': 14
}

ann_num_to_str = {}
for k, v in ann_str_to_num.items():
    if v == 5:
        k = '(N\x00'
    elif v == 12:
        k = '(VF\x00'
    ann_num_to_str[v] = k


VFDB_LOCATION = './vfdb/'
QRS_LOCATION = './vfdb-qrs/'
VFDB_RECORDS = [418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428,
                429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]

def load_qrs_i(record):
    """Returns a list with the positions of Rs that pan_tompkins returned for
       a specific record.
    """
    filepath = QRS_LOCATION + record + '_1s_i.txt'
    qrs_i = numpy.genfromtxt(filepath, delimiter=',')
    return qrs_i

def load_qrs_amp(record):
    """Returns a list with the amp values that pan_tompkins returned for
       a specific record.
    """
    filepath = QRS_LOCATION + record + '_1s_amp.txt'
    qrs_amp = numpy.genfromtxt(filepath, delimiter=',')
    return qrs_amp

def get_annotation(record):
    """Returns the annotatation object for a given record
    """
    record_path = VFDB_LOCATION + record
    ann = wfdb.rdann(record_path, 'atr')
    return ann

def get_record(record):
    """Returns the annotatation object for a given record
    """
    record_path = VFDB_LOCATION + record
    record = wfdb.rdrecord(record_path)
    return record

def get_annotation_positions(ann):
    """ Takes an annotation object and returns a list of tuples:
        [
        ('<idx if annotaion position>', 'annotations aux_label'),
        ('<idx if annotaion position>', 'annotations aux_label'),
        ...
        ('<idx if annotaion position>', 'annotations aux_label')
        ]
    """
    return list(zip(ann.sample.tolist(), ann.aux_note))

def create_qrs_annotation_mapping(qrs_i, qrs_amp, ann_pos):
    """Assign annotations to every R peak.

    :param qrs_i: list with the r positions
    :param qrs_amp: amp values in the qrs_i positions
    :param ann_pos: list of tuples with the position of every annotation

    returns a ndarray (number of R, 3) Every row will contain the
            index of the R pick, the amp of the peak and the annotation
            as number. See the dict in the beggining of this file for the
            mapping.
            [
                [index_of_R_peak, value_of_amp, annotation_id],
                ...
                [index_of_R_peak, value_of_amp, annotation_id]
            ]
    """
    # From 0 to the first annotation set the annotation to normal.
    ann_pos = [(0, '(N\x00')] + ann_pos
    # number of annotation
    len_ann_pos = len(ann_pos)
    # Number of R peaks
    len_qrs_i = len(qrs_i)
    # Initialize a zeros array for the result
    res = numpy.zeros((len_qrs_i, 3))
    # For every annotation loop the R peaks
    i = 0
    while i < len_ann_pos:
        # Take the current annotation string and its position in the siganl
        pos, ann = ann_pos[i]
        # Take the posotion of the next annotation
        ann = ann_str_to_num[ann]
        # In the case of the last annotation loop until the end of the R peaks
        # array.
        if i == len_ann_pos - 1:
            next_pos = qrs_i[qrs_i.size - 1] + 1
        else:
            next_pos, next_ann = ann_pos[i + 1]
        # For evert peak:
        x = 0
        while x < len_qrs_i:
            # If the R index is after the index of the annotation and
            # before the next annotation (ot the end of the R array)
            # set the annotation string
            if pos <= qrs_i[x] and qrs_i[x] < next_pos:
                res[x][0] = qrs_i[x]
                res[x][1] = qrs_amp[x]
                # if the annotation is NOISE a.k.a 7, set the previous
                # annotation
                if ann == 7 and pos != 0:
                    ann = ann_str_to_num[ann_pos[i - 1][1]]
                res[x][2] = ann
            x = x +1
        i = i + 1

    return res

def create_rr_annotation(annotated_r_peaks):
    """Create RR segments and annotate them.

    :param qrs_i: np.array with the r positions
    :param ann_pos: list of tuples with the position of every annotation

    returns a length(qrs_i) by 4 array which looks like this:
    [
        [start_or_RR_segment, end_or_RR_segnemnt, RR_distane (aka end_or_RR_segnemnt - start_or_RR_segment), annotation_id]
        ...
        [start_or_RR_segment, end_or_RR_segnemnt, RR_distane (aka end_or_RR_segnemnt - start_or_RR_segment), annotation_id]
    ]
    """
    res = numpy.zeros((annotated_r_peaks[:,0].size - 1, 4))
    # Set the start of the RR segments
    # Take all R indices
    all_r_idx = annotated_r_peaks[:,0]
    # Take the annotations for all Rs
    all_annots = annotated_r_peaks[:,2]

    # For the start R take from the first to the second to last
    # position 0 to -1
    start_r = all_r_idx[0:-1]
    # For the end R take from the second to the end, position 1 to end
    end_r = all_r_idx[1:all_r_idx.size]
    if start_r.size != end_r.size:
        raise Exception("start_r and end_r have diffrent sizes.")
    # The 1st column is the start R peak
    res[:,0] = start_r
    # The 2nd column is the end R peak
    res[:,1] = end_r
    # The 3rd column is the difference of end - start Rs
    res[:,2] = end_r - start_r
    # The 4th column is the annotation of the RR segment, defined by the
    # annotation of the end R.
    res[:,3] = all_annots[1:all_annots.size]

    return res

def save_rr_segment_to_csv(record_id):
    """
    Takes a record id from vfdb and writes the RR segment with its annotation
    to a csv file.

    :param record_id: The record ID eg 418
    """

    qrs_i = load_qrs_i(record_id)
    #print type(qrs_i)
    qrs_amp = load_qrs_amp(record_id)
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annotated_r_peaks = create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
    annotated_rr_segments = create_rr_annotation(annotated_r_peaks)
    # numpy.set_printoptions(suppress=True)
    # print annotated_rr_segments
    numpy.savetxt(QRS_LOCATION + record_id + '_RR.txt', annotated_rr_segments, delimiter=",")


In [4]:
for record in VFDB_RECORDS:
    print(record, len(load_qrs_i(str(record))))

(418, 4301)
(419, 3906)
(420, 2924)
(421, 4526)
(422, 4073)
(423, 3538)
(424, 3796)
(425, 2129)
(426, 3775)
(427, 3895)
(428, 3266)
(429, 3614)
(430, 4696)
(602, 3308)
(605, 1922)
(607, 1848)
(609, 2215)
(610, 2167)
(611, 3948)
(612, 2490)
(614, 3330)
(615, 1593)


In [7]:
ann_str_to_num  = {
    '(AFIB\x00': 0,
    '(ASYS\x00': 1,
    '(B\x00': 2,
    '(BI\x00': 3,
    '(HGEA\x00': 4,
    '(N\x00': 5,
    '(NSR\x00': 5,
    '(NOD\x00': 6,
    '(NOISE\x00': 7,
    '(PM\x00': 8,
    '(SBR\x00': 9,
    '(SVTA\x00': 10,
    '(VER\x00': 11,
    '(VF\x00': 12,
    '(VFIB\x00': 12,
    '(VFL\x00': 13,
    '(VT\x00': 14
}

ann_num_to_str = {}
for k, v in ann_str_to_num.items():
    if v == 5:
        k = '(N\x00'
    elif v == 12:
        k = '(VF\x00'
    ann_num_to_str[v] = k
for item in ann_num_to_str.items():
    print (item)

(0, '(AFIB\x00')
(1, '(ASYS\x00')
(2, '(B\x00')
(3, '(BI\x00')
(4, '(HGEA\x00')
(5, '(N\x00')
(6, '(NOD\x00')
(7, '(NOISE\x00')
(8, '(PM\x00')
(9, '(SBR\x00')
(10, '(SVTA\x00')
(11, '(VER\x00')
(12, '(VF\x00')
(13, '(VFL\x00')
(14, '(VT\x00')


In [9]:
print (VFDB_RECORDS)

[418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]


In [12]:
import numpy
print ("\nrecord \t num_of_RR")
for record in VFDB_RECORDS:
    filepath = QRS_LOCATION + str(record) + '_RR.csv'
    rr = numpy.genfromtxt(filepath, delimiter=',')
    print (record, '\t', rr[:,0].size)


record 	 num_of_RR
418 	 4300
419 	 3905
420 	 2923
421 	 4525
422 	 4072
423 	 3537
424 	 3795
425 	 2128
426 	 3774
427 	 3894
428 	 3265
429 	 3613
430 	 4695
602 	 3307
605 	 1921
607 	 1847
609 	 2214
610 	 2166
611 	 3947
612 	 2489
614 	 3329
615 	 1592


In [8]:
# Find all different labels in every annotation
import pprint

records = [str(x) for x in VFDB_RECORDS]
for record in records:
    ann = get_annotation(record)
    #print record
    #pprint.pprint(set(ann.aux_note))
    labels = set(ann.aux_note)
    labels = list(labels)
    print (record, sorted(labels))

('418', ['(N\x00', '(VFL\x00'])
('419', ['(N\x00', '(NOISE\x00', '(VFL\x00'])
('420', ['(N\x00', '(NOISE\x00', '(VT\x00'])
('421', ['(N\x00', '(NOISE\x00', '(VT\x00'])
('422', ['(N\x00', '(NOISE\x00', '(VFIB\x00', '(VT\x00'])
('423', ['(AFIB\x00', '(ASYS\x00', '(N\x00', '(NOD\x00', '(NOISE\x00', '(VT\x00'])
('424', ['(ASYS\x00', '(N\x00', '(NOD\x00', '(NOISE\x00', '(NSR\x00', '(VFIB\x00', '(VFL\x00'])
('425', ['(B\x00', '(N\x00', '(NOISE\x00', '(VT\x00'])
('426', ['(N\x00', '(NOISE\x00', '(SVTA\x00', '(VF\x00', '(VT\x00'])
('427', ['(ASYS\x00', '(N\x00', '(VT\x00'])
('428', ['(BI\x00', '(NOISE\x00', '(VT\x00'])
('429', ['(BI\x00', '(NOISE\x00', '(VFL\x00', '(VT\x00'])
('430', ['(ASYS\x00', '(BI\x00', '(HGEA\x00', '(NOISE\x00', '(SBR\x00', '(VER\x00', '(VF\x00', '(VFL\x00', '(VT\x00'])
('602', ['(ASYS\x00', '(N\x00', '(NOISE\x00', '(PM\x00', '(VT\x00'])
('605', ['(NOISE\x00', '(VT\x00'])
('607', ['(ASYS\x00', '(NOD\x00', '(SVTA\x00', '(VT\x00'])
('609', ['(AFIB\x00', '(HGEA\x00', '(N\x0

In [26]:
from collections import Counter
import numpy
import pprint
#import vfdb_annotate_r

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 100

for record in records:
    filepath = QRS_LOCATION + str(record) + '_RR.csv'
    rrs = numpy.genfromtxt(filepath, delimiter=',')
    rows, columns = rrs.shape
    num_segments = rows / (SEGMENT_LEN / 2)
    i = 0
    l = []
    while i < num_segments:
        start = int(i * (SEGMENT_LEN / 2))
        if i == num_segments - 1:
            end = rows - 1
        else:
            end = start + SEGMENT_LEN
        # start is the idx of the first RR of this segment
        # end is the idx of the last RR in this segment
        
        labels = rrs[start:end, 3]
        data = Counter(labels)
        most_frequent = max(data)
        if data[most_frequent] >= 0.8 * SEGMENT_LEN:
            l.append(ann_num_to_str[most_frequent])
        else:
            l.append("NO_LABEL")
                
        #print [ann_num_to_str[x] for x in list(set(labels))]            
        i += 1
    print ("record", record, "has", num_segments, "segments", Counter(l))

('record', '418', 'has', 86, 'segments', Counter({'NO_LABEL': 49, '(N\x00': 37}))
('record', '419', 'has', 78, 'segments', Counter({'(VFL\x00': 34, '(N\x00': 26, 'NO_LABEL': 18}))
('record', '420', 'has', 58, 'segments', Counter({'(N\x00': 31, '(VT\x00': 25, 'NO_LABEL': 2}))
('record', '421', 'has', 90, 'segments', Counter({'(N\x00': 57, '(VT\x00': 19, 'NO_LABEL': 14}))
('record', '422', 'has', 81, 'segments', Counter({'(N\x00': 44, '(VF\x00': 24, '(VT\x00': 10, 'NO_LABEL': 3}))
('record', '423', 'has', 70, 'segments', Counter({'(N\x00': 23, '(VT\x00': 23, '(AFIB\x00': 19, 'NO_LABEL': 5}))
('record', '424', 'has', 75, 'segments', Counter({'(N\x00': 58, 'NO_LABEL': 10, '(VF\x00': 5, '(VFL\x00': 2}))
('record', '425', 'has', 42, 'segments', Counter({'(N\x00': 27, 'NO_LABEL': 8, '(B\x00': 5, '(VT\x00': 2}))
('record', '426', 'has', 75, 'segments', Counter({'(VF\x00': 42, '(N\x00': 15, 'NO_LABEL': 14, '(VT\x00': 4}))
('record', '427', 'has', 77, 'segments', Counter({'(VT\x00': 44, '(ASYS\x

In [5]:
# Drop the first 10 RRs
# group in segments of 10 RR
# print and write in file:
# record number, [RR-0, RR-1, RR-2 ... RR-9], annotation string
# example:
# ['418', array([189., 140., 134., 135., 133.,  89.,  83., 115.,  83., 167.]), 5.0]

from collections import Counter
import numpy
import pprint
import csv

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 10
segments_fp =  QRS_LOCATION + 'segments.csv'

with open(segments_fp, 'w') as fh:
    for record in records:
        filepath = QRS_LOCATION + str(record) + '_RR.csv'
        rrs = numpy.genfromtxt(filepath, delimiter=',')
        rows, columns = rrs.shape
        #num_segments = rows / (SEGMENT_LEN / 2)
        num_segments = int(rows / SEGMENT_LEN)
        # start from the 2nd segment
        i = 1
        l = []
        while i < num_segments - 1:
            start = i * SEGMENT_LEN
            end = start + SEGMENT_LEN
            labels = rrs[start:end, 3]
            current_rrs = rrs[start:end, 2]
            data = Counter(labels)
            most_frequent = max(data)
            if data[most_frequent] == SEGMENT_LEN:
                entry = [record, current_rrs, most_frequent]
                print(entry)
                writer = csv.writer(fh)
                writer.writerow(entry)
                
            i += 1

['418', array([189., 140., 134., 135., 133.,  89.,  83., 115.,  83., 167.]), 5.0]
['418', array([138.,  82., 186., 136.,  88., 178., 135.,  84., 182., 141.]), 5.0]
['418', array([136., 134., 137., 136., 135.,  78., 188., 139., 139., 136.]), 5.0]
['418', array([135., 136., 132., 137.,  83., 183., 140.,  87., 182., 138.]), 5.0]
['418', array([135., 137., 134., 135., 136.,  81., 190., 136., 134., 136.]), 5.0]
['418', array([136., 138.,  78.,  84.,  78., 163., 140.,  80.,  78., 112.]), 5.0]
['418', array([142., 131.,  82., 190., 138., 139., 134., 137., 133., 134.]), 5.0]
['418', array([135.,  87., 187., 138.,  81., 186., 139.,  80., 187., 139.]), 5.0]
['418', array([ 81., 191., 139.,  78., 189., 139.,  88., 186., 139.,  79.]), 5.0]
['418', array([186., 138.,  78., 188., 139.,  81., 188., 142.,  77., 194.]), 5.0]
['418', array([141.,  79., 190., 139.,  80., 187., 139.,  78., 190., 175.]), 5.0]
['418', array([ 41., 192., 139.,  80., 195., 142.,  77., 191., 139.,  78.]), 5.0]
['418', array([1

['419', array([194., 145.,  82., 197., 142.,  87., 190., 145.,  82., 194.]), 5.0]
['419', array([142.,  83., 199., 141.,  82., 190., 142.,  82., 200., 142.]), 5.0]
['419', array([ 83., 194., 145.,  81., 196., 144.,  84., 193., 141.,  83.]), 5.0]
['419', array([196., 140.,  81., 200., 143.,  82., 195., 142.,  82., 199.]), 5.0]
['419', array([140.,  82., 197., 143.,  83., 199., 144., 141.,  89.,  78.]), 5.0]
['419', array([ 79., 169., 145.,  82., 195., 143.,  79., 195., 141.,  82.]), 5.0]
['419', array([ 45., 163., 139.,  83.,  85., 157.,  74.,  77., 114., 103.]), 5.0]
['419', array([151., 140., 141., 142., 140., 142., 136., 140., 138., 141.]), 5.0]
['419', array([143., 140., 140., 138.,  86., 192., 140.,  82., 200., 140.]), 5.0]
['419', array([ 82., 191., 143.,  81., 203., 143.,  76.,  77.,  72.,  96.]), 5.0]
['419', array([ 80., 196., 141., 145.,  88., 191., 139.,  80., 198., 141.]), 5.0]
['419', array([141., 141., 139., 143., 144.,  93., 130., 193., 142.,  80.]), 5.0]
['419', array([1

['420', array([231., 229., 231., 228., 232., 227., 226., 232., 226., 226.]), 5.0]
['420', array([224., 230., 233., 180., 264., 234., 233., 238., 226., 227.]), 5.0]
['420', array([227., 229., 230., 228., 232., 223., 234., 227., 226., 225.]), 5.0]
['420', array([227., 233., 235., 233., 221., 219., 225., 221., 226., 246.]), 5.0]
['420', array([236., 245., 215., 217., 216., 224., 224., 218., 223., 228.]), 5.0]
['420', array([217., 220., 236., 220., 215., 230., 228., 222., 234., 233.]), 5.0]
['420', array([222., 237., 235., 242., 228., 239., 225., 230., 224., 219.]), 5.0]
['420', array([217., 227., 225., 221., 225., 230., 227., 221., 230., 224.]), 5.0]
['420', array([219., 219., 232., 223., 222., 224., 223., 223., 230., 226.]), 5.0]
['420', array([223., 226., 221., 234., 216., 223., 216., 223., 228., 222.]), 5.0]
['420', array([223., 227., 228., 221., 231., 226., 215., 225., 235., 215.]), 5.0]
['420', array([223., 230., 218., 225., 227., 223., 219., 235., 223., 163.]), 5.0]
['420', array([2

['421', array([150., 140., 150., 140., 150., 140., 150., 140., 140., 150.]), 5.0]
['421', array([150., 140., 150., 150., 140., 150., 150., 140., 150., 150.]), 5.0]
['421', array([140., 150., 150., 150., 140., 150., 150., 150., 150., 150.]), 5.0]
['421', array([150., 150., 150., 150., 150., 150., 150., 150., 150., 180.]), 5.0]
['421', array([ 80.,  40., 150., 150., 150., 140., 150., 150., 140., 150.]), 5.0]
['421', array([150., 160., 140., 160., 140., 150., 150., 160., 150., 150.]), 5.0]
['421', array([150., 150., 150., 150., 150., 150., 150., 150., 150., 150.]), 5.0]
['421', array([150., 150., 150., 160., 150., 150., 140., 150., 150., 150.]), 5.0]
['421', array([150., 150., 150., 150., 140., 150., 150., 140., 150., 150.]), 5.0]
['421', array([140., 150., 140., 150., 150., 150., 150., 140., 150., 140.]), 5.0]
['421', array([150., 150., 140., 150., 150., 150., 140., 150., 150., 150.]), 5.0]
['421', array([150., 150., 150., 140., 150., 160., 140., 150., 150., 160.]), 5.0]
['421', array([1

['422', array([153., 157., 154., 151., 154., 152., 151., 156., 156., 157.]), 5.0]
['422', array([152., 148., 153., 151., 157., 159., 160., 156., 155., 155.]), 5.0]
['422', array([160., 152., 156., 156., 155., 153., 152., 150., 151., 151.]), 5.0]
['422', array([151., 148., 147., 149., 144., 148., 146., 148., 145., 147.]), 5.0]
['422', array([148., 147., 151., 148., 144., 144., 148., 146., 145., 148.]), 5.0]
['422', array([147., 148., 145., 146., 144., 149., 149., 148., 149., 142.]), 5.0]
['422', array([147., 149., 148., 152., 149., 148., 148., 151., 147., 154.]), 5.0]
['422', array([150., 151., 152., 147., 153., 152., 151., 156., 153., 151.]), 5.0]
['422', array([152., 151., 155., 152., 156., 152., 153., 154., 151., 152.]), 5.0]
['422', array([151., 150., 154., 152., 146., 150., 152., 150., 152., 149.]), 5.0]
['422', array([148., 149., 149., 149., 152., 147., 147., 151., 147., 147.]), 5.0]
['422', array([151., 151., 145., 150., 146., 149., 150., 140., 150., 160.]), 5.0]
['422', array([1

...

['607', array([340., 360., 350., 360., 340., 350., 340., 350., 340., 340.]), 6.0]
['607', array([340., 340., 340., 330., 340., 340., 330., 340., 340., 340.]), 6.0]
['607', array([330., 340., 340., 330., 340., 330., 360., 350., 360., 350.]), 6.0]
['607', array([340., 350., 340., 330., 340., 360., 360., 320., 370., 330.]), 6.0]
['607', array([340., 360., 350., 340., 330., 350., 330., 340., 340., 350.]), 6.0]
['607', array([360., 320., 340., 340., 330., 340., 330., 340., 340., 340.]), 6.0]
['607', array([330., 340., 330., 340., 360., 350., 360., 340., 350., 340.]), 6.0]
['607', array([340., 350., 350., 340., 340., 350., 340., 340., 340., 330.]), 6.0]
['607', array([340., 340., 340., 290., 290., 280., 150., 100., 300., 170.]), 6.0]
['607', array([140., 340., 160., 370., 350., 310., 260., 260., 260., 250.]), 6.0]
['607', array([260., 260., 250., 250., 360., 370., 360., 350., 360., 340.]), 6.0]
['607', array([350., 350., 350., 350., 340., 360., 340., 350., 340., 340.]), 6.0]
['607', array([3

['610', array([330., 300., 310., 300., 300., 310., 300., 300., 260., 330.]), 5.0]
['610', array([140., 360., 190., 100., 370., 280., 130., 350., 280., 110.]), 4.0]
['610', array([280., 160., 100., 270., 250., 290., 150.,  90.,  80., 350.]), 4.0]
['610', array([160., 110., 100., 290., 160.,  90.,  90., 390., 170., 330.]), 4.0]
['610', array([190., 110., 360., 270., 170., 100., 260., 180., 380., 160.]), 4.0]
['610', array([100.,  60.,  80., 390., 190., 350., 170., 280., 370., 180.]), 4.0]
['610', array([100., 100., 340., 280.,  80.,  80.,  80., 390., 400., 160.]), 4.0]
['610', array([310., 190., 100., 350., 200., 310., 190., 100., 370., 200.]), 4.0]
['610', array([110., 360., 180., 100., 170., 320., 200., 330., 200., 320.]), 4.0]
['610', array([200., 330., 210., 330., 200., 160., 300., 190., 330., 190.]), 4.0]
['610', array([100., 330., 200., 320., 290., 330., 190., 140., 340., 270.]), 4.0]
['610', array([ 80., 120., 310., 270., 150., 280., 150., 350., 180., 110.]), 4.0]
['610', array([3

['611', array([160., 150., 160., 160., 160., 160., 160., 150., 160., 150.]), 10.0]
['611', array([160., 160., 160., 150., 170., 140., 160., 150., 170., 140.]), 10.0]
['611', array([180., 150., 170., 140., 180., 150., 160., 150., 170., 160.]), 10.0]
['611', array([160., 150., 160., 160., 160., 150., 160., 160., 150., 160.]), 10.0]
['611', array([160., 150., 160., 160., 160., 150., 160., 160., 160., 150.]), 10.0]
['611', array([160., 160., 160., 150., 160., 150., 160., 160., 150., 160.]), 10.0]
['611', array([160., 150., 160., 160., 160., 150., 160., 160., 160., 150.]), 10.0]
['611', array([160., 150., 160., 160., 160., 160., 150., 160., 160., 160.]), 10.0]
['611', array([150., 160., 150., 160., 150., 160., 160., 150., 150., 160.]), 10.0]
['611', array([160., 160., 150., 160., 150., 160., 150., 160., 160., 150.]), 10.0]
['611', array([160., 160., 150., 160., 150., 160., 160., 150., 160., 160.]), 10.0]
['611', array([150., 160., 160., 160., 150., 160., 150., 170., 150., 160.]), 10.0]
['61

['612', array([381., 165., 401., 156., 385., 279., 271., 287., 300., 305.]), 5.0]
['612', array([346., 153., 185., 281., 148., 200., 301., 142., 190., 276.]), 5.0]
['612', array([145., 198., 264., 152., 196., 281., 146., 196., 281., 145.]), 5.0]
['612', array([208., 301., 145., 214., 314., 147., 202., 277., 150., 197.]), 5.0]
['612', array([297., 147., 191., 288., 150., 209., 319., 146., 214., 331.]), 5.0]
['612', array([144., 190., 271., 331., 324., 325., 323., 306., 314., 320.]), 5.0]
['612', array([300., 318., 322., 327., 313., 337., 328., 320., 331., 332.]), 5.0]
['612', array([333., 336., 339., 339., 341., 292., 310., 310., 319., 324.]), 5.0]
['612', array([296., 313., 320., 307., 330., 275., 291., 291., 294., 291.]), 5.0]
['612', array([276., 279., 281., 279., 293., 286., 280., 303., 284., 290.]), 5.0]
['612', array([295., 289., 300., 294., 316., 291., 304., 290., 286., 289.]), 5.0]
['612', array([300., 321., 332., 330., 335., 330., 325., 340., 347., 327.]), 5.0]
['612', array([3

['614', array([130., 300., 140., 220., 130., 280., 140.,  80., 360., 140.]), 2.0]
['614', array([70., 70., 60., 60., 70., 60., 70., 60., 70., 70.]), 14.0]
['614', array([ 60.,  90.,  70.,  80., 110.,  70.,  70.,  60., 110.,  90.]), 14.0]
['614', array([160.,  60.,  70., 130.,  50., 270.,  40.,  40.,  50.,  60.]), 14.0]
['614', array([110.,  60.,  70., 110., 120.,  70.,  60.,  90.,  60., 120.]), 14.0]
['614', array([130.,  50.,  60., 110.,  60., 100.,  70.,  60.,  60.,  60.]), 14.0]
['614', array([130., 110.,  50.,  60.,  90.,  60.,  60., 130., 110.,  50.]), 14.0]
['614', array([ 70.,  60.,  50., 100.,  30., 110.,  60., 110.,  40., 130.]), 14.0]
['614', array([100., 120.,  60.,  70.,  60.,  60., 100.,  50., 100.,  60.]), 14.0]
['614', array([ 60.,  50., 120.,  60., 220.,  90., 210.,  50.,  60., 100.]), 14.0]
['614', array([150., 110.,  30.,  60., 240.,  80.,  60., 110., 100., 170.]), 14.0]
['614', array([100.,  60., 160.,  60.,  60.,  50.,  50.,  50.,  60.,  60.]), 14.0]
['614', array([

In [ ]:
# Drop the first 10 RRs
# group in segments of 10 RR
# print and write in file:
# record number, [RR-0, RR-1, RR-2 ... RR-9], annotation string


# Save without the entry id, only the 10 values of the segment and
# the most frequent label.
# example:
# [189.0, 140.0, 134.0, 135.0, 133.0, 89.0, 83.0, 115.0, 83.0, 167.0, 5.0]

from collections import Counter
import numpy
import pprint
import csv

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 10
segments_fp =  QRS_LOCATION + 'segments_new.csv'

with open(segments_fp, 'w') as fh:
    for record in records:
        filepath = QRS_LOCATION + str(record) + '_RR.csv'
        rrs = numpy.genfromtxt(filepath, delimiter=',')
        rows, columns = rrs.shape
        #num_segments = rows / (SEGMENT_LEN / 2)
        num_segments = int(rows / SEGMENT_LEN)
        # start from the 2nd segment
        i = 1
        l = []
        while i < num_segments - 1:
            start = i * SEGMENT_LEN
            end = start + SEGMENT_LEN
            labels = rrs[start:end, 3]
            current_rrs = rrs[start:end, 2]
            data = Counter(labels)
            most_frequent = max(data)
            if data[most_frequent] == SEGMENT_LEN:
                #entry = [record, current_rrs, most_frequent]
                entry = current_rrs.tolist()
                entry.append(most_frequent)
                print(entry)
                writer = csv.writer(fh)
                writer.writerow(entry)
                
            i += 1

[189.0, 140.0, 134.0, 135.0, 133.0, 89.0, 83.0, 115.0, 83.0, 167.0, 5.0]
[138.0, 82.0, 186.0, 136.0, 88.0, 178.0, 135.0, 84.0, 182.0, 141.0, 5.0]
[136.0, 134.0, 137.0, 136.0, 135.0, 78.0, 188.0, 139.0, 139.0, 136.0, 5.0]
[135.0, 136.0, 132.0, 137.0, 83.0, 183.0, 140.0, 87.0, 182.0, 138.0, 5.0]
[135.0, 137.0, 134.0, 135.0, 136.0, 81.0, 190.0, 136.0, 134.0, 136.0, 5.0]
[136.0, 138.0, 78.0, 84.0, 78.0, 163.0, 140.0, 80.0, 78.0, 112.0, 5.0]
[142.0, 131.0, 82.0, 190.0, 138.0, 139.0, 134.0, 137.0, 133.0, 134.0, 5.0]
[135.0, 87.0, 187.0, 138.0, 81.0, 186.0, 139.0, 80.0, 187.0, 139.0, 5.0]
[81.0, 191.0, 139.0, 78.0, 189.0, 139.0, 88.0, 186.0, 139.0, 79.0, 5.0]
[186.0, 138.0, 78.0, 188.0, 139.0, 81.0, 188.0, 142.0, 77.0, 194.0, 5.0]
[141.0, 79.0, 190.0, 139.0, 80.0, 187.0, 139.0, 78.0, 190.0, 175.0, 5.0]
[41.0, 192.0, 139.0, 80.0, 195.0, 142.0, 77.0, 191.0, 139.0, 78.0, 5.0]
[192.0, 140.0, 80.0, 193.0, 137.0, 77.0, 188.0, 139.0, 79.0, 194.0, 5.0]
[139.0, 80.0, 189.0, 140.0, 80.0, 188.0, 142.0, 

[130.0, 130.0, 140.0, 130.0, 130.0, 130.0, 130.0, 140.0, 80.0, 180.0, 5.0]
[130.0, 140.0, 130.0, 130.0, 80.0, 180.0, 130.0, 90.0, 180.0, 130.0, 5.0]
[140.0, 130.0, 130.0, 130.0, 140.0, 120.0, 90.0, 180.0, 130.0, 130.0, 5.0]
[130.0, 130.0, 140.0, 130.0, 130.0, 130.0, 90.0, 170.0, 140.0, 130.0, 5.0]
[130.0, 130.0, 140.0, 130.0, 130.0, 130.0, 130.0, 130.0, 130.0, 140.0, 5.0]
[130.0, 130.0, 130.0, 130.0, 130.0, 140.0, 130.0, 130.0, 130.0, 130.0, 5.0]
[130.0, 130.0, 140.0, 130.0, 130.0, 130.0, 130.0, 140.0, 130.0, 130.0, 5.0]
[90.0, 80.0, 70.0, 80.0, 90.0, 80.0, 80.0, 80.0, 70.0, 80.0, 13.0]
[80.0, 140.0, 70.0, 110.0, 80.0, 80.0, 70.0, 100.0, 90.0, 70.0, 13.0]
[80.0, 70.0, 80.0, 80.0, 70.0, 80.0, 90.0, 70.0, 80.0, 80.0, 13.0]
[80.0, 80.0, 80.0, 70.0, 80.0, 90.0, 80.0, 120.0, 90.0, 80.0, 13.0]
[120.0, 120.0, 130.0, 90.0, 170.0, 130.0, 140.0, 130.0, 130.0, 90.0, 5.0]
[170.0, 130.0, 130.0, 130.0, 140.0, 80.0, 180.0, 130.0, 130.0, 130.0, 5.0]
[130.0, 90.0, 170.0, 130.0, 140.0, 130.0, 130.0, 80.

[79.0, 169.0, 145.0, 82.0, 195.0, 143.0, 79.0, 195.0, 141.0, 82.0, 5.0]
[45.0, 163.0, 139.0, 83.0, 85.0, 157.0, 74.0, 77.0, 114.0, 103.0, 5.0]
[151.0, 140.0, 141.0, 142.0, 140.0, 142.0, 136.0, 140.0, 138.0, 141.0, 5.0]
[143.0, 140.0, 140.0, 138.0, 86.0, 192.0, 140.0, 82.0, 200.0, 140.0, 5.0]
[82.0, 191.0, 143.0, 81.0, 203.0, 143.0, 76.0, 77.0, 72.0, 96.0, 5.0]
[80.0, 196.0, 141.0, 145.0, 88.0, 191.0, 139.0, 80.0, 198.0, 141.0, 5.0]
[141.0, 141.0, 139.0, 143.0, 144.0, 93.0, 130.0, 193.0, 142.0, 80.0, 5.0]
[142.0, 140.0, 140.0, 140.0, 140.0, 143.0, 140.0, 140.0, 75.0, 197.0, 5.0]
[145.0, 80.0, 199.0, 142.0, 139.0, 144.0, 139.0, 141.0, 139.0, 77.0, 5.0]
[198.0, 142.0, 80.0, 201.0, 140.0, 141.0, 136.0, 140.0, 141.0, 139.0, 5.0]
[80.0, 198.0, 143.0, 140.0, 137.0, 141.0, 141.0, 81.0, 198.0, 144.0, 5.0]
[140.0, 138.0, 140.0, 110.0, 169.0, 142.0, 138.0, 138.0, 137.0, 143.0, 5.0]
[142.0, 137.0, 136.0, 138.0, 174.0, 90.0, 187.0, 104.0, 144.0, 170.0, 5.0]
[137.0, 135.0, 107.0, 143.0, 117.0, 194.0

[80.0, 100.0, 90.0, 60.0, 170.0, 320.0, 110.0, 90.0, 90.0, 30.0, 13.0]
[190.0, 150.0, 140.0, 230.0, 60.0, 50.0, 100.0, 100.0, 80.0, 150.0, 13.0]
[110.0, 40.0, 80.0, 100.0, 90.0, 150.0, 40.0, 110.0, 190.0, 290.0, 13.0]
[120.0, 140.0, 230.0, 100.0, 110.0, 90.0, 250.0, 150.0, 180.0, 50.0, 13.0]
[100.0, 50.0, 90.0, 170.0, 90.0, 90.0, 110.0, 90.0, 100.0, 100.0, 13.0]
[90.0, 140.0, 130.0, 90.0, 60.0, 170.0, 170.0, 100.0, 50.0, 140.0, 13.0]
[80.0, 30.0, 150.0, 90.0, 140.0, 170.0, 160.0, 140.0, 80.0, 30.0, 13.0]
[150.0, 180.0, 50.0, 100.0, 210.0, 180.0, 50.0, 100.0, 100.0, 100.0, 13.0]
[80.0, 200.0, 140.0, 90.0, 170.0, 140.0, 90.0, 30.0, 170.0, 140.0, 13.0]
[140.0, 160.0, 150.0, 130.0, 120.0, 50.0, 120.0, 50.0, 160.0, 80.0, 13.0]
[180.0, 100.0, 90.0, 60.0, 290.0, 140.0, 80.0, 110.0, 100.0, 110.0, 13.0]
[100.0, 120.0, 120.0, 130.0, 90.0, 60.0, 120.0, 150.0, 90.0, 120.0, 13.0]
[210.0, 140.0, 80.0, 100.0, 120.0, 140.0, 90.0, 190.0, 140.0, 90.0, 13.0]
[60.0, 120.0, 160.0, 150.0, 180.0, 50.0, 80.0,

[231.0, 229.0, 231.0, 228.0, 232.0, 227.0, 226.0, 232.0, 226.0, 226.0, 5.0]
[224.0, 230.0, 233.0, 180.0, 264.0, 234.0, 233.0, 238.0, 226.0, 227.0, 5.0]
[227.0, 229.0, 230.0, 228.0, 232.0, 223.0, 234.0, 227.0, 226.0, 225.0, 5.0]
[227.0, 233.0, 235.0, 233.0, 221.0, 219.0, 225.0, 221.0, 226.0, 246.0, 5.0]
[236.0, 245.0, 215.0, 217.0, 216.0, 224.0, 224.0, 218.0, 223.0, 228.0, 5.0]
[217.0, 220.0, 236.0, 220.0, 215.0, 230.0, 228.0, 222.0, 234.0, 233.0, 5.0]
[222.0, 237.0, 235.0, 242.0, 228.0, 239.0, 225.0, 230.0, 224.0, 219.0, 5.0]
[217.0, 227.0, 225.0, 221.0, 225.0, 230.0, 227.0, 221.0, 230.0, 224.0, 5.0]
[219.0, 219.0, 232.0, 223.0, 222.0, 224.0, 223.0, 223.0, 230.0, 226.0, 5.0]
[223.0, 226.0, 221.0, 234.0, 216.0, 223.0, 216.0, 223.0, 228.0, 222.0, 5.0]
[223.0, 227.0, 228.0, 221.0, 231.0, 226.0, 215.0, 225.0, 235.0, 215.0, 5.0]
[223.0, 230.0, 218.0, 225.0, 227.0, 223.0, 219.0, 235.0, 223.0, 163.0, 5.0]
[252.0, 258.0, 233.0, 230.0, 230.0, 220.0, 223.0, 226.0, 230.0, 223.0, 5.0]
[233.0, 224.

...

[320.0, 330.0, 360.0, 360.0, 350.0, 350.0, 370.0, 360.0, 360.0, 340.0, 5.0]
[340.0, 350.0, 360.0, 340.0, 240.0, 250.0, 180.0, 190.0, 100.0, 100.0, 5.0]
[221.0, 226.0, 230.0, 218.0, 226.0, 230.0, 216.0, 223.0, 228.0, 217.0, 5.0]
[233.0, 233.0, 217.0, 225.0, 221.0, 214.0, 214.0, 219.0, 212.0, 222.0, 5.0]
[221.0, 214.0, 226.0, 227.0, 216.0, 219.0, 219.0, 216.0, 224.0, 216.0, 5.0]
[212.0, 229.0, 228.0, 218.0, 224.0, 235.0, 217.0, 219.0, 217.0, 212.0, 5.0]
[218.0, 227.0, 216.0, 222.0, 228.0, 217.0, 231.0, 231.0, 223.0, 231.0, 5.0]
[234.0, 218.0, 224.0, 222.0, 216.0, 228.0, 233.0, 216.0, 228.0, 226.0, 5.0]
[214.0, 216.0, 225.0, 216.0, 222.0, 224.0, 218.0, 223.0, 232.0, 217.0, 5.0]
[222.0, 227.0, 219.0, 226.0, 221.0, 216.0, 221.0, 231.0, 216.0, 218.0, 5.0]
[223.0, 216.0, 230.0, 222.0, 214.0, 217.0, 223.0, 213.0, 212.0, 220.0, 5.0]
[215.0, 218.0, 219.0, 214.0, 220.0, 222.0, 215.0, 219.0, 216.0, 212.0, 5.0]
[215.0, 220.0, 213.0, 222.0, 226.0, 223.0, 218.0, 220.0, 219.0, 218.0, 5.0]
[227.0, 213.

[110.0, 120.0, 80.0, 170.0, 100.0, 80.0, 140.0, 100.0, 150.0, 110.0, 14.0]
[90.0, 110.0, 100.0, 110.0, 120.0, 90.0, 150.0, 100.0, 150.0, 100.0, 14.0]
[120.0, 140.0, 90.0, 140.0, 80.0, 120.0, 140.0, 90.0, 150.0, 80.0, 14.0]
[80.0, 110.0, 90.0, 90.0, 110.0, 130.0, 130.0, 140.0, 90.0, 140.0, 14.0]
[90.0, 80.0, 150.0, 50.0, 280.0, 80.0, 90.0, 90.0, 200.0, 50.0, 14.0]
[40.0, 80.0, 90.0, 90.0, 80.0, 100.0, 130.0, 100.0, 90.0, 140.0, 14.0]
[50.0, 40.0, 90.0, 80.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 14.0]
[40.0, 190.0, 90.0, 120.0, 100.0, 140.0, 130.0, 130.0, 330.0, 140.0, 14.0]
[70.0, 50.0, 40.0, 80.0, 180.0, 140.0, 120.0, 50.0, 90.0, 130.0, 14.0]
[50.0, 90.0, 40.0, 100.0, 50.0, 80.0, 170.0, 90.0, 90.0, 90.0, 14.0]
[90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 80.0, 100.0, 90.0, 80.0, 14.0]
[80.0, 150.0, 80.0, 90.0, 120.0, 130.0, 90.0, 90.0, 150.0, 80.0, 10.0]
[160.0, 100.0, 100.0, 90.0, 150.0, 50.0, 80.0, 80.0, 90.0, 50.0, 14.0]
[80.0, 60.0, 70.0, 60.0, 60.0, 60.0, 70.0, 60.0, 70.0, 60.0, 14.0]
[80.

[70.0, 50.0, 120.0, 60.0, 70.0, 100.0, 40.0, 60.0, 120.0, 100.0, 14.0]
[70.0, 30.0, 110.0, 30.0, 220.0, 100.0, 90.0, 100.0, 130.0, 190.0, 14.0]
[100.0, 180.0, 110.0, 150.0, 60.0, 90.0, 80.0, 40.0, 50.0, 210.0, 14.0]
[100.0, 70.0, 70.0, 60.0, 70.0, 100.0, 100.0, 140.0, 130.0, 140.0, 14.0]
[120.0, 120.0, 100.0, 270.0, 110.0, 130.0, 70.0, 70.0, 150.0, 60.0, 14.0]
[160.0, 150.0, 110.0, 60.0, 140.0, 60.0, 80.0, 120.0, 90.0, 100.0, 14.0]
[110.0, 100.0, 120.0, 100.0, 190.0, 60.0, 210.0, 60.0, 90.0, 90.0, 14.0]
[200.0, 190.0, 40.0, 80.0, 130.0, 90.0, 170.0, 40.0, 80.0, 100.0, 14.0]
[60.0, 120.0, 140.0, 60.0, 60.0, 70.0, 60.0, 60.0, 120.0, 70.0, 14.0]
[110.0, 100.0, 50.0, 240.0, 130.0, 160.0, 40.0, 160.0, 90.0, 70.0, 14.0]
[220.0, 200.0, 90.0, 160.0, 760.0, 120.0, 390.0, 160.0, 70.0, 90.0, 1.0]
[280.0, 300.0, 160.0, 530.0, 310.0, 330.0, 510.0, 110.0, 280.0, 220.0, 1.0]
[160.0, 140.0, 450.0, 470.0, 260.0, 260.0, 170.0, 690.0, 170.0, 120.0, 1.0]
[170.0, 380.0, 130.0, 210.0, 520.0, 150.0, 340.0, 3

[160.0, 150.0, 90.0, 190.0, 150.0, 80.0, 200.0, 140.0, 110.0, 170.0, 3.0]
[160.0, 110.0, 170.0, 150.0, 110.0, 170.0, 150.0, 110.0, 170.0, 150.0, 3.0]
[90.0, 190.0, 150.0, 110.0, 160.0, 150.0, 90.0, 190.0, 150.0, 110.0, 3.0]
[160.0, 150.0, 90.0, 90.0, 120.0, 140.0, 140.0, 100.0, 180.0, 140.0, 3.0]
[154.0, 153.0, 152.0, 152.0, 152.0, 155.0, 155.0, 154.0, 115.0, 186.0, 3.0]
[160.0, 154.0, 154.0, 120.0, 183.0, 157.0, 157.0, 152.0, 120.0, 172.0, 3.0]
[150.0, 153.0, 151.0, 152.0, 175.0, 128.0, 151.0, 153.0, 153.0, 151.0, 3.0]
[152.0, 149.0, 153.0, 154.0, 153.0, 151.0, 156.0, 155.0, 146.0, 148.0, 3.0]
[129.0, 169.0, 160.0, 159.0, 147.0, 113.0, 176.0, 155.0, 155.0, 149.0, 3.0]
[154.0, 156.0, 155.0, 151.0, 117.0, 173.0, 159.0, 154.0, 152.0, 152.0, 3.0]
[146.0, 153.0, 158.0, 155.0, 150.0, 148.0, 148.0, 149.0, 148.0, 150.0, 3.0]
[153.0, 116.0, 189.0, 159.0, 150.0, 146.0, 121.0, 167.0, 154.0, 176.0, 3.0]
[126.0, 148.0, 152.0, 153.0, 154.0, 153.0, 150.0, 150.0, 149.0, 150.0, 3.0]
[149.0, 154.0, 149

[70.0, 60.0, 60.0, 60.0, 60.0, 60.0, 70.0, 190.0, 30.0, 130.0, 13.0]
[40.0, 80.0, 60.0, 70.0, 120.0, 40.0, 60.0, 100.0, 60.0, 70.0, 13.0]
[60.0, 60.0, 80.0, 80.0, 120.0, 60.0, 130.0, 50.0, 100.0, 140.0, 13.0]
[70.0, 60.0, 120.0, 60.0, 110.0, 70.0, 110.0, 60.0, 60.0, 60.0, 13.0]
[130.0, 210.0, 60.0, 70.0, 110.0, 40.0, 100.0, 100.0, 100.0, 120.0, 13.0]
[40.0, 140.0, 60.0, 70.0, 160.0, 60.0, 110.0, 70.0, 70.0, 70.0, 13.0]
[70.0, 70.0, 70.0, 70.0, 90.0, 40.0, 70.0, 70.0, 60.0, 100.0, 13.0]
[40.0, 100.0, 70.0, 30.0, 80.0, 60.0, 80.0, 100.0, 70.0, 30.0, 13.0]
[70.0, 70.0, 70.0, 70.0, 60.0, 100.0, 70.0, 40.0, 90.0, 80.0, 13.0]
[70.0, 30.0, 70.0, 70.0, 70.0, 100.0, 40.0, 70.0, 70.0, 90.0, 13.0]
[40.0, 70.0, 70.0, 90.0, 70.0, 30.0, 150.0, 100.0, 60.0, 70.0, 13.0]
[40.0, 100.0, 70.0, 100.0, 70.0, 100.0, 60.0, 40.0, 70.0, 100.0, 13.0]
[70.0, 30.0, 100.0, 40.0, 70.0, 70.0, 70.0, 90.0, 140.0, 60.0, 13.0]
[40.0, 100.0, 60.0, 40.0, 70.0, 100.0, 40.0, 90.0, 80.0, 60.0, 13.0]
[70.0, 70.0, 60.0, 140.0, 

In [6]:
# Drop the first 10 RRs
# group in segments of 10 RR
# print and write in file:
# record number, [RR-0, RR-1, RR-2 ... RR-9], annotation string
# example:
# ['418', array([189., 140., 134., 135., 133.,  89.,  83., 115.,  83., 167.]), 5.0]

from collections import Counter
import numpy
import pprint
import csv

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 10
segments_fp =  QRS_LOCATION + 'segments_new.csv'

with open(segments_fp, 'w') as fh:
    for record in records:
        filepath = QRS_LOCATION + str(record) + '_RR.csv'
        rrs = numpy.genfromtxt(filepath, delimiter=',')
        rows, columns = rrs.shape
        #num_segments = rows / (SEGMENT_LEN / 2)
        num_segments = int(rows / SEGMENT_LEN)
        # start from the 2nd segment
        i = 1
        l = []
        while i < num_segments - 1:
            start = i * SEGMENT_LEN
            end = start + SEGMENT_LEN
            labels = rrs[start:end, 3]
            current_rrs = rrs[start:end, 2]
            data = Counter(labels)
            most_frequent = max(data)
            if data[most_frequent] == SEGMENT_LEN:
                #entry = [record, current_rrs, most_frequent]
                entry = current_rrs.tolist()
                entry.append(most_frequent)
                print(entry)
                writer = csv.writer(fh)
                writer.writerow(entry)
                
            i += 1

[189.0, 140.0, 134.0, 135.0, 133.0, 89.0, 83.0, 115.0, 83.0, 167.0, 5.0]
[138.0, 82.0, 186.0, 136.0, 88.0, 178.0, 135.0, 84.0, 182.0, 141.0, 5.0]
[136.0, 134.0, 137.0, 136.0, 135.0, 78.0, 188.0, 139.0, 139.0, 136.0, 5.0]
[135.0, 136.0, 132.0, 137.0, 83.0, 183.0, 140.0, 87.0, 182.0, 138.0, 5.0]
[135.0, 137.0, 134.0, 135.0, 136.0, 81.0, 190.0, 136.0, 134.0, 136.0, 5.0]
[136.0, 138.0, 78.0, 84.0, 78.0, 163.0, 140.0, 80.0, 78.0, 112.0, 5.0]
[142.0, 131.0, 82.0, 190.0, 138.0, 139.0, 134.0, 137.0, 133.0, 134.0, 5.0]
[135.0, 87.0, 187.0, 138.0, 81.0, 186.0, 139.0, 80.0, 187.0, 139.0, 5.0]
[81.0, 191.0, 139.0, 78.0, 189.0, 139.0, 88.0, 186.0, 139.0, 79.0, 5.0]
[186.0, 138.0, 78.0, 188.0, 139.0, 81.0, 188.0, 142.0, 77.0, 194.0, 5.0]
[141.0, 79.0, 190.0, 139.0, 80.0, 187.0, 139.0, 78.0, 190.0, 175.0, 5.0]
[41.0, 192.0, 139.0, 80.0, 195.0, 142.0, 77.0, 191.0, 139.0, 78.0, 5.0]
[192.0, 140.0, 80.0, 193.0, 137.0, 77.0, 188.0, 139.0, 79.0, 194.0, 5.0]
[139.0, 80.0, 189.0, 140.0, 80.0, 188.0, 142.0, 

[130.0, 150.0, 110.0, 130.0, 120.0, 130.0, 130.0, 90.0, 40.0, 90.0, 5.0]
[60.0, 120.0, 130.0, 150.0, 110.0, 150.0, 110.0, 130.0, 90.0, 160.0, 5.0]
[110.0, 160.0, 140.0, 130.0, 130.0, 130.0, 130.0, 130.0, 140.0, 130.0, 5.0]
[140.0, 130.0, 130.0, 140.0, 130.0, 130.0, 140.0, 130.0, 140.0, 140.0, 5.0]
[130.0, 140.0, 130.0, 130.0, 140.0, 130.0, 140.0, 130.0, 140.0, 130.0, 5.0]
[140.0, 130.0, 140.0, 130.0, 60.0, 70.0, 140.0, 140.0, 130.0, 130.0, 5.0]
[140.0, 130.0, 50.0, 220.0, 140.0, 40.0, 90.0, 150.0, 50.0, 60.0, 5.0]
[140.0, 130.0, 130.0, 130.0, 140.0, 130.0, 130.0, 140.0, 90.0, 40.0, 5.0]
[140.0, 90.0, 180.0, 70.0, 70.0, 140.0, 130.0, 130.0, 160.0, 110.0, 5.0]
[160.0, 40.0, 70.0, 140.0, 130.0, 140.0, 130.0, 140.0, 130.0, 140.0, 5.0]
[140.0, 130.0, 140.0, 130.0, 130.0, 140.0, 150.0, 120.0, 50.0, 150.0, 5.0]
[120.0, 90.0, 140.0, 70.0, 60.0, 100.0, 100.0, 80.0, 130.0, 160.0, 5.0]
[110.0, 140.0, 130.0, 140.0, 180.0, 90.0, 100.0, 170.0, 140.0, 130.0, 5.0]
[90.0, 320.0, 140.0, 90.0, 70.0, 110.

[90.0, 130.0, 150.0, 120.0, 130.0, 120.0, 110.0, 110.0, 110.0, 110.0, 0.0]
[130.0, 120.0, 130.0, 130.0, 110.0, 130.0, 120.0, 90.0, 90.0, 140.0, 0.0]
[110.0, 160.0, 110.0, 150.0, 110.0, 150.0, 110.0, 110.0, 140.0, 140.0, 0.0]
[100.0, 120.0, 100.0, 140.0, 110.0, 120.0, 110.0, 90.0, 150.0, 130.0, 0.0]
[120.0, 140.0, 140.0, 100.0, 110.0, 80.0, 110.0, 130.0, 170.0, 100.0, 0.0]
[140.0, 100.0, 110.0, 120.0, 100.0, 170.0, 90.0, 120.0, 120.0, 120.0, 0.0]
[140.0, 130.0, 110.0, 120.0, 110.0, 110.0, 120.0, 110.0, 90.0, 100.0, 0.0]
[130.0, 90.0, 100.0, 90.0, 120.0, 150.0, 150.0, 110.0, 110.0, 180.0, 0.0]
[130.0, 110.0, 140.0, 120.0, 130.0, 120.0, 80.0, 160.0, 100.0, 90.0, 0.0]
[98.0, 93.0, 123.0, 99.0, 96.0, 113.0, 106.0, 90.0, 148.0, 137.0, 5.0]
[134.0, 142.0, 120.0, 101.0, 158.0, 142.0, 155.0, 123.0, 120.0, 124.0, 5.0]
[126.0, 94.0, 138.0, 103.0, 90.0, 121.0, 112.0, 138.0, 108.0, 104.0, 5.0]
[108.0, 177.0, 102.0, 86.0, 98.0, 88.0, 52.0, 153.0, 122.0, 95.0, 5.0]
[169.0, 110.0, 95.0, 88.0, 84.0, 91

[236.0, 223.0, 167.0, 234.0, 203.0, 211.0, 204.0, 202.0, 205.0, 203.0, 5.0]
[145.0, 249.0, 225.0, 212.0, 207.0, 163.0, 241.0, 208.0, 152.0, 247.0, 5.0]
[215.0, 208.0, 207.0, 200.0, 201.0, 201.0, 204.0, 200.0, 199.0, 198.0, 5.0]
[204.0, 200.0, 194.0, 200.0, 199.0, 201.0, 199.0, 197.0, 203.0, 202.0, 5.0]
[200.0, 203.0, 202.0, 207.0, 207.0, 205.0, 206.0, 209.0, 206.0, 201.0, 5.0]
[205.0, 209.0, 207.0, 201.0, 203.0, 208.0, 208.0, 199.0, 202.0, 201.0, 5.0]
[204.0, 199.0, 197.0, 200.0, 202.0, 198.0, 195.0, 199.0, 199.0, 160.0, 5.0]
[235.0, 205.0, 203.0, 153.0, 242.0, 207.0, 151.0, 252.0, 209.0, 204.0, 5.0]
[208.0, 208.0, 146.0, 244.0, 223.0, 211.0, 207.0, 206.0, 204.0, 209.0, 5.0]
[206.0, 203.0, 204.0, 188.0, 222.0, 203.0, 198.0, 206.0, 198.0, 199.0, 5.0]
[201.0, 198.0, 202.0, 199.0, 196.0, 188.0, 213.0, 201.0, 196.0, 200.0, 5.0]
[202.0, 203.0, 198.0, 201.0, 208.0, 205.0, 199.0, 203.0, 205.0, 209.0, 5.0]
[206.0, 204.0, 207.0, 207.0, 203.0, 202.0, 208.0, 203.0, 207.0, 203.0, 5.0]
[205.0, 209.

[150.0, 144.0, 84.0, 196.0, 146.0, 140.0, 84.0, 191.0, 143.0, 143.0, 3.0]
[82.0, 196.0, 144.0, 139.0, 82.0, 178.0, 154.0, 106.0, 113.0, 112.0, 3.0]
[54.0, 151.0, 158.0, 143.0, 145.0, 85.0, 198.0, 142.0, 138.0, 83.0, 3.0]
[194.0, 148.0, 140.0, 84.0, 194.0, 142.0, 143.0, 111.0, 167.0, 144.0, 3.0]
[143.0, 113.0, 53.0, 108.0, 151.0, 139.0, 139.0, 87.0, 195.0, 148.0, 3.0]
[139.0, 113.0, 147.0, 135.0, 130.0, 89.0, 167.0, 134.0, 132.0, 121.0, 3.0]
[138.0, 131.0, 131.0, 89.0, 170.0, 131.0, 140.0, 85.0, 173.0, 130.0, 3.0]
[131.0, 123.0, 131.0, 134.0, 141.0, 117.0, 202.0, 142.0, 132.0, 121.0, 3.0]
[135.0, 133.0, 135.0, 121.0, 148.0, 134.0, 151.0, 109.0, 182.0, 145.0, 3.0]
[137.0, 120.0, 56.0, 91.0, 134.0, 131.0, 140.0, 114.0, 193.0, 151.0, 3.0]
[145.0, 85.0, 192.0, 136.0, 139.0, 116.0, 175.0, 144.0, 141.0, 85.0, 3.0]
[189.0, 147.0, 138.0, 90.0, 193.0, 148.0, 141.0, 88.0, 193.0, 145.0, 3.0]
[140.0, 91.0, 181.0, 153.0, 141.0, 88.0, 188.0, 143.0, 141.0, 88.0, 3.0]
[182.0, 132.0, 134.0, 123.0, 159.0

[230.0, 240.0, 240.0, 230.0, 240.0, 230.0, 230.0, 230.0, 240.0, 230.0, 5.0]
[220.0, 230.0, 230.0, 240.0, 230.0, 230.0, 240.0, 230.0, 240.0, 230.0, 5.0]
[230.0, 250.0, 240.0, 220.0, 220.0, 260.0, 250.0, 230.0, 250.0, 230.0, 5.0]
[230.0, 230.0, 230.0, 240.0, 230.0, 230.0, 240.0, 220.0, 250.0, 230.0, 5.0]
[230.0, 240.0, 230.0, 250.0, 240.0, 230.0, 240.0, 230.0, 240.0, 230.0, 5.0]
[240.0, 230.0, 240.0, 240.0, 230.0, 230.0, 210.0, 210.0, 280.0, 240.0, 5.0]
[250.0, 230.0, 220.0, 230.0, 230.0, 240.0, 240.0, 240.0, 250.0, 240.0, 5.0]
[220.0, 220.0, 220.0, 220.0, 220.0, 230.0, 220.0, 220.0, 220.0, 230.0, 5.0]
[220.0, 220.0, 230.0, 210.0, 230.0, 230.0, 220.0, 220.0, 220.0, 230.0, 5.0]
[220.0, 220.0, 230.0, 220.0, 230.0, 220.0, 230.0, 230.0, 220.0, 220.0, 5.0]
[230.0, 230.0, 230.0, 230.0, 230.0, 210.0, 240.0, 220.0, 230.0, 220.0, 5.0]
[230.0, 230.0, 230.0, 230.0, 230.0, 220.0, 230.0, 230.0, 230.0, 230.0, 5.0]
[220.0, 230.0, 230.0, 230.0, 230.0, 230.0, 220.0, 240.0, 220.0, 220.0, 5.0]
[240.0, 230.

[170.0, 170.0, 170.0, 240.0, 180.0, 180.0, 240.0, 180.0, 170.0, 240.0, 5.0]
[180.0, 260.0, 180.0, 270.0, 180.0, 170.0, 170.0, 240.0, 180.0, 250.0, 5.0]
[180.0, 240.0, 180.0, 160.0, 230.0, 180.0, 170.0, 220.0, 170.0, 240.0, 5.0]
[180.0, 220.0, 190.0, 170.0, 160.0, 160.0, 170.0, 150.0, 150.0, 230.0, 5.0]
[170.0, 160.0, 230.0, 170.0, 150.0, 240.0, 190.0, 220.0, 200.0, 190.0, 5.0]
[170.0, 150.0, 160.0, 160.0, 150.0, 150.0, 160.0, 140.0, 150.0, 140.0, 5.0]
[150.0, 140.0, 160.0, 150.0, 150.0, 150.0, 140.0, 150.0, 150.0, 140.0, 5.0]
[140.0, 260.0, 230.0, 200.0, 190.0, 190.0, 180.0, 180.0, 150.0, 150.0, 5.0]
[140.0, 150.0, 150.0, 140.0, 150.0, 230.0, 160.0, 150.0, 160.0, 250.0, 5.0]
[180.0, 180.0, 150.0, 150.0, 160.0, 150.0, 150.0, 150.0, 150.0, 150.0, 5.0]
[150.0, 140.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 140.0, 230.0, 5.0]
[170.0, 160.0, 150.0, 150.0, 150.0, 140.0, 150.0, 140.0, 150.0, 150.0, 5.0]
[140.0, 140.0, 150.0, 150.0, 140.0, 150.0, 140.0, 150.0, 150.0, 140.0, 5.0]
[260.0, 170.

In [22]:
record_id='418'
qrs_i = load_qrs_i(record_id)
#print type(qrs_i)
qrs_amp = load_qrs_amp(record_id)
ann = get_annotation(record_id)
annot_positions = get_annotation_positions(ann)
annotated_r_peaks = create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
annotated_rr_segments = create_rr_annotation(annotated_r_peaks)
print(annotated_rr_segments)

[[2.4000e+01 1.2800e+02 1.0400e+02 5.0000e+00]
 [1.2800e+02 2.6100e+02 1.3300e+02 5.0000e+00]
 [2.6100e+02 3.4500e+02 8.4000e+01 5.0000e+00]
 ...
 [5.2460e+05 5.2473e+05 1.3000e+02 5.0000e+00]
 [5.2473e+05 5.2486e+05 1.3000e+02 5.0000e+00]
 [5.2486e+05 5.2499e+05 1.3000e+02 5.0000e+00]]


In [30]:
# for every record print label episode!
for record in VFDB_RECORDS:
    record_id=str(record)
    record_data=get_record(record_id)
    record_size=record_data.sig_len
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annot_positions = [(0, '(N\x00')] + annot_positions
    print ('record:', record_id)
    i=0
    while i<len(annot_positions):
        if i==0: start=i
        else: start=annot_positions[i][0] + 1
        if i==len(annot_positions)-1:
            end=record_size-1
        else:
            end=annot_positions[i+1][0]
        print (start,end,annot_positions[i][1])
        i+=1    

record: 418
0 18 (N 
19 99624 (N 
99625 101499 (VFL 
101500 133092 (N 
133093 134038 (VFL 
134039 135775 (N 
135776 136628 (VFL 
136629 153057 (N 
153058 154115 (VFL 
154116 154942 (N 
154943 156291 (VFL 
156292 159442 (N 
159443 160516 (VFL 
160517 169192 (N 
169193 169807 (VFL 
169808 173054 (N 
173055 173673 (VFL 
173674 174788 (N 
174789 175403 (VFL 
175404 176259 (N 
176260 177868 (VFL 
177869 190080 (N 
190081 191249 (VFL 
191250 191807 (N 
191808 192695 (VFL 
192696 195631 (N 
195632 196794 (VFL 
196795 200211 (N 
200212 200634 (VFL 
200635 216788 (N 
216789 219038 (VFL 
219039 219961 (N 
219962 224019 (VFL 
224020 225355 (N 
225356 226057 (VFL 
226058 227211 (N 
227212 229269 (VFL 
229270 231310 (N 
231311 232724 (VFL 
232725 234499 (N 
234500 235538 (VFL 
235539 254230 (N 
254231 255365 (VFL 
255366 256019 (N 
256020 256884 (VFL 
256885 257249 (N 
257250 257980 (VFL 
257981 259557 (N 
259558 261903 (VFL 
261904 262749 (N 
262750 263519 (VFL 
263520 269307 (N 
269308 270999 (VF

record: 430
0 28 (N 
29 51903 (BI 
51904 53211 (VT 
53212 85615 (BI 
85616 93134 (NOISE 
93135 94423 (BI 
94424 94903 (VT 
94904 98596 (BI 
98597 99557 (BI 
99558 118057 (VFL 
118058 120807 (NOISE 
120808 131153 (VF 
131154 134519 (ASYS 
134520 138076 (SBR 
138077 156576 (HGEA 
156577 157923 (VT 
157924 163192 (HGEA 
163193 169179 (VT 
169180 170237 (ASYS 
170238 172275 (HGEA 
172276 173891 (VT 
173892 175429 (HGEA 
175430 193891 (VF 
193892 196217 (ASYS 
196218 273294 (VF 
273295 274391 (ASYS 
274392 310544 (VF 
310545 315333 (ASYS 
315334 319871 (SBR 
319872 406794 (VF 
406795 407102 (ASYS 
407103 415025 (VER 
415026 489621 (VT 
489622 493794 (VER 
493795 524999 (VF 
record: 602
0 64 (N 
65 121275 (PM 
121276 121756 (VT 
121757 286814 (PM 
286815 287333 (VT 
287334 289352 (PM 
289353 289910 (VT 
289911 311833 (PM 
311834 408525 (VT 
408526 411044 (ASYS 
411045 422487 (N 
422488 467987 (NOISE 
467988 524999 (PM 
record: 605
0 64 (N 
65 411942 (NOISE 
411943 419115 (VT 
419116 524999 (

In [31]:
#print the number of R in each episode
for record in VFDB_RECORDS:
    record_id=str(record)
    print ('record:', record_id)
    qrs_i = load_qrs_i(record_id)
    #print type(qrs_i)
    qrs_amp = load_qrs_amp(record_id)
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annotated_r_peaks = create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
    i=1
    annot=annotated_r_peaks[0][2]
    s=1
    while i<annotated_r_peaks[:,0].size:
        if annotated_r_peaks[i-1][2]==annotated_r_peaks[i][2]:
            s+=1
        else:
            print(ann_num_to_str[annotated_r_peaks[i-1][2]], '\t', s)
            s=1
        i+=1

record: 418
(N  	 748
(VFL  	 22
(N  	 240
(VFL  	 12
(N  	 17
(VFL  	 11
(N  	 128
(VFL  	 11
(N  	 9
(VFL  	 17
(N  	 26
(VFL  	 11
(N  	 69
(VFL  	 5
(N  	 26
(VFL  	 8
(N  	 9
(VFL  	 6
(N  	 7
(VFL  	 21
(N  	 96
(VFL  	 16
(N  	 4
(VFL  	 12
(N  	 23
(VFL  	 11
(N  	 26
(VFL  	 3
(N  	 136
(VFL  	 22
(N  	 6
(VFL  	 48
(N  	 11
(VFL  	 7
(N  	 9
(VFL  	 23
(N  	 17
(VFL  	 17
(N  	 14
(VFL  	 12
(N  	 152
(VFL  	 13
(N  	 5
(VFL  	 11
(N  	 4
(VFL  	 11
(N  	 12
(VFL  	 25
(N  	 7
(VFL  	 7
(N  	 46
(VFL  	 25
(N  	 3
(VFL  	 1
(N  	 4
(VFL  	 4
(N  	 11
(VFL  	 26
(N  	 28
(VFL  	 23
(N  	 61
(VFL  	 23
(N  	 70
(VFL  	 8
(N  	 3
(VFL  	 2
(N  	 5
(VFL  	 3
(N  	 3
(VFL  	 4
(N  	 2
(VFL  	 12
(N  	 52
(VFL  	 4
(N  	 2
(VFL  	 17
(N  	 2
(VFL  	 6
(N  	 3
(VFL  	 27
(N  	 4
(VFL  	 3
(N  	 9
(VFL  	 12
(N  	 59
(VFL  	 18
(N  	 3
(VFL  	 3
(N  	 35
(VFL  	 5
(N  	 43
(VFL  	 3
(N  	 51
(VFL  	 4
(N  	 3
(VFL  	 3
(N  	 75
(VFL  	 8
(N  	 33
(VFL  	 5
(N  	 35
(VFL  	 4
(N  	 3


In [26]:
# for every record print the number of each label
ans = ""
for k in ann_str_to_num.keys():
    ans = ans + " " + str(k)
print(ans)

for record in VFDB_RECORDS:
    record_id=str(record)
    record_data=get_record(record_id)
    record_size=record_data.sig_len
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annot_positions = [(0, '(N\x00')] + annot_positions
    #print ('record: ' + str(record_id))
    i=0
    d = {}
    while i<len(annot_positions):
        if annot_positions[i][1] in d:
            d[annot_positions[i][1]] = d[annot_positions[i][1]] + 1
        else:
            d[annot_positions[i][1]] = 1
        i+=1
    #print(d.keys())
    ans = ""
    for k in ann_str_to_num.keys():
        if k in d:
            ans = ans + " " + str(d[k])
        else:
            ans = ans + " 0"
    #print('record: ' + str(record_id) + " " + ans)
    print(ans)

            #print(str(k) + " " + str(d[k]))
    #for k,v in d.items():
    #    print(str(k) + " " + str(v))

 (VF  (N  (HGEA  (ASYS  (NOD  (NSR  (B  (NOISE  (SVTA  (BI  (SBR  (VT  (VFIB  (VER  (PM  (VFL  (AFIB 
 0 62 0 0 0 0 0 0 0 0 0 0 0 0 0 60 0
 0 31 0 0 0 0 0 6 0 0 0 0 0 0 0 33 0
 0 3 0 0 0 0 0 4 0 0 0 3 0 0 0 0 0
 0 68 0 0 0 0 0 17 0 0 0 50 0 0 0 0 0
 0 12 0 0 0 0 0 13 0 0 0 2 3 0 0 0 0
 0 5 0 1 1 0 0 3 0 0 0 1 0 0 0 0 2
 0 2 0 3 2 2 0 4 0 0 0 0 1 0 0 1 0
 0 10 0 0 0 0 2 8 0 0 0 1 0 0 0 0 0
 3 4 0 0 0 0 0 4 3 0 0 4 0 0 0 0 0
 0 2 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 5 0 3 0 0 0 0 0
 0 1 0 0 0 0 0 6 0 12 0 3 0 0 0 2 0
 6 1 4 6 0 0 0 2 0 5 2 6 0 2 0 1 0
 0 2 0 1 0 0 0 1 0 0 0 4 0 0 5 0 0
 0 1 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0
 0 1 0 1 3 0 0 0 2 0 0 4 0 0 0 0 0
 0 5 4 0 0 0 0 0 0 0 0 1 0 1 0 1 1
 0 13 11 0 0 0 0 2 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 10 0 0 0 0 10 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 5 0 0 0 0 6


In [29]:
for record in VFDB_RECORDS:
    print('"'+str(record)+'"')

"418"
"419"
"420"
"421"
"422"
"423"
"424"
"425"
"426"
"427"
"428"
"429"
"430"
"602"
"605"
"607"
"609"
"610"
"611"
"612"
"614"
"615"


In [34]:
ans = ""
for k in ann_str_to_num.keys():
    ans = ans + " " + str(k)
print(ans)

# for every record print label episode!
for record in VFDB_RECORDS:
    record_id=str(record)
    record_data=get_record(record_id)
    record_size=record_data.sig_len
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annot_positions = [(0, '(N\x00')] + annot_positions
    #print ('record:', record_id)
    i=0
    d = {}
    
    while i<len(annot_positions):
        if i==0: start=i
        else: start=annot_positions[i][0] + 1
        if i==len(annot_positions)-1:
            end=record_size-1
        else:
            end=annot_positions[i+1][0]
        if annot_positions[i][1] in d:
            d[annot_positions[i][1]] = d[annot_positions[i][1]] + (end - start)
        else:
            d[annot_positions[i][1]] = (end - start)
        #print (start,end,annot_positions[i][1])
        i+=1
    ans = ""
    for k in ann_str_to_num.keys():
        if k in d:
            ans = ans + " " + str(d[k])
        else:
            ans = ans + " 0"
    #print('record: ' + str(record_id) + " " + ans)
    print(ans)

 (VF  (N  (HGEA  (ASYS  (NOD  (NSR  (B  (NOISE  (SVTA  (BI  (SBR  (VT  (VFIB  (VER  (PM  (VFL  (AFIB 
 0 463425 0 0 0 0 0 0 0 0 0 0 0 0 0 61453 0
 0 278908 0 0 0 0 0 193927 0 0 0 0 0 0 0 52095 0
 0 330055 0 0 0 0 0 124419 0 0 0 70516 0 0 0 0 0
 0 330086 0 0 0 0 0 108444 0 0 0 86335 0 0 0 0 0
 0 293702 0 0 0 0 0 52581 0 0 0 48767 129920 0 0 0 0
 0 258957 0 2557 5883 0 0 140746 0 0 0 113903 0 0 0 0 2941
 0 314748 0 16325 28151 54787 0 70090 0 0 0 0 26423 0 0 14461 0
 0 361096 0 0 0 0 81462 75223 0 0 0 7198 0 0 0 0 0
 162084 208348 0 0 0 0 0 120342 20347 0 0 13861 0 0 0 0 0
 0 162123 0 141739 0 0 0 0 0 0 0 221134 0 0 0 0 0
 0 38 0 0 0 0 0 2864 0 510533 0 11555 0 0 0 0 0
 0 41 0 0 0 0 0 47609 0 451984 0 2382 0 0 0 22960 0
 260160 28 27341 12938 0 0 0 10267 0 90217 8093 85328 0 12094 0 18499 0
 0 11506 0 2518 0 0 0 45499 0 0 0 98246 0 0 367218 0 0
 0 64 0 0 0 0 0 517760 0 0 0 7172 0 0 0 0 0
 0 38 0 21922 406747 0 0 0 57709 0 0 38573 0 0 0 0 0
 0 218194 42970 0 0 0 0 0 0 0 0 1595 0 1614 0 53

In [43]:
# create segments of 5 RR

#AFIB
#B-VT-VF-VFL-ASYS
#BI
#HGEA-N-NOD-PM-SBR-VER
#SVTA

map_classes_14_to_5 = {
    ann_str_to_num['(AFIB\x00']: 'AFIB',
    ann_str_to_num['(ASYS\x00']: 'B-VT-VF-VFL-ASYS',
    ann_str_to_num['(B\x00']: 'B-VT-VF-VFL-ASYS',
    ann_str_to_num['(BI\x00']: 'BI',
    ann_str_to_num['(HGEA\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(N\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(NSR\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(NOD\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(NOISE\x00']: 'NOISE',
    ann_str_to_num['(PM\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(SBR\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(SVTA\x00']: '',
    ann_str_to_num['(VER\x00']: 'HGEA-N-NOD-PM-SBR-VER',
    ann_str_to_num['(VF\x00']: 'B-VT-VF-VFL-ASYS',
    ann_str_to_num['(VFIB\x00']: 'B-VT-VF-VFL-ASYS',
    ann_str_to_num['(VFL\x00']: 'B-VT-VF-VFL-ASYS',
    ann_str_to_num['(VT\x00']: 'B-VT-VF-VFL-ASYS'   
}


# Drop the first 10 RRs
# group in segments of 10 RR
# print and write in file:
# record number, [RR-0, RR-1, RR-2 ... RR-9], annotation string
# example:
# ['418', array([189., 140., 134., 135., 133.,  89.,  83., 115.,  83., 167.]), 5.0]

from collections import Counter
import numpy
import pprint
import csv

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 10
segments_fp =  QRS_LOCATION + 'segments_5RR.csv'
segments_fp_5_classes =  QRS_LOCATION + 'segments_5RR_5_classes.csv'    

def run(fh, my_mapping):
    for record in records:
        filepath = QRS_LOCATION + str(record) + '_RR.csv'
        rrs = numpy.genfromtxt(filepath, delimiter=',')
        rows, columns = rrs.shape
        #num_segments = rows / (SEGMENT_LEN / 2)
        num_segments = int(rows / SEGMENT_LEN)
        # start from the 2nd segment
        i = 1
        l = []
        while i < num_segments - 1:
            start = i * SEGMENT_LEN
            end = start + SEGMENT_LEN
            labels = rrs[start:end, 3]
            current_rrs = rrs[start:end, 2]
            data = Counter(labels)
            most_frequent = max(data)
            if data[most_frequent] == SEGMENT_LEN:
                entry = [record, current_rrs, most_frequent]
                entry1 = str(current_rrs[0:5]) + ", " + my_mapping[most_frequent]
                entry2 = str(current_rrs[5:10]) + ", " + my_mapping[most_frequent]
                print(entry1)
                print(entry2)
                writer = csv.writer(fh)
                writer.writerow(entry1)
                writer.writerow(entry2)
            i += 1
            
with open(segments_fp, 'w') as fh:
    run(fh, ann_num_to_str)
    
with open(segments_fp_5_classes, 'w') as fh:
    run(fh, map_classes_14_to_5)

[189. 140. 134. 135. 133.], (N 
[ 89.  83. 115.  83. 167.], (N 
[138.  82. 186. 136.  88.], (N 
[178. 135.  84. 182. 141.], (N 
[136. 134. 137. 136. 135.], (N 
[ 78. 188. 139. 139. 136.], (N 
[135. 136. 132. 137.  83.], (N 
[183. 140.  87. 182. 138.], (N 
[135. 137. 134. 135. 136.], (N 
[ 81. 190. 136. 134. 136.], (N 
[136. 138.  78.  84.  78.], (N 
[163. 140.  80.  78. 112.], (N 
[142. 131.  82. 190. 138.], (N 
[139. 134. 137. 133. 134.], (N 
[135.  87. 187. 138.  81.], (N 
[186. 139.  80. 187. 139.], (N 
[ 81. 191. 139.  78. 189.], (N 
[139.  88. 186. 139.  79.], (N 
[186. 138.  78. 188. 139.], (N 
[ 81. 188. 142.  77. 194.], (N 
[141.  79. 190. 139.  80.], (N 
[187. 139.  78. 190. 175.], (N 
[ 41. 192. 139.  80. 195.], (N 
[142.  77. 191. 139.  78.], (N 
[192. 140.  80. 193. 137.], (N 
[ 77. 188. 139.  79. 194.], (N 
[139.  80. 189. 140.  80.], (N 
[188. 142.  81. 194. 141.], (N 
[ 79. 190. 142.  79.  75.], (N 
[118. 136. 135. 137. 137.], (N 
[137. 138. 137. 133.  82.], (N 
[185. 14

[150. 140. 150. 140. 150.], (N 
[ 80. 200. 150. 150. 150.], (N 
[140. 150. 140. 150. 150.], (N 
[140. 150. 130. 160. 150.], (N 
[140. 150. 140. 150. 140.], (N 
[ 90. 200. 150. 150. 140.], (N 
[150. 150. 140. 150. 150.], (N 
[140. 150. 140. 150. 150.], (N 
[140. 150. 140.  90. 200.], (N 
[150. 140. 150. 140. 150.], (N 
[150. 140. 150. 140. 150.], (N 
[140. 150. 150. 140. 150.], (N 
[140. 140. 150.  80. 210.], (N 
[150. 140. 140. 150. 150.], (N 
[140.  90. 150. 200. 140.], (N 
[ 60.  60. 150.  80.  60.], (VFL 
[ 60.  60. 110.  60.  60.], (VFL 
[ 60. 180. 230.  60. 120.], (VFL 
[ 60.  60. 120.  60.  60.], (VFL 
[120.  60. 120. 110. 120.], (VFL 
[ 60.  60. 120.  60.  60.], (VFL 
[ 50.  60. 120.  60.  60.], (VFL 
[110.  60.  60. 120.  60.], (VFL 
[60. 60. 60. 60. 70.], (VFL 
[60. 60. 60. 60. 60.], (VFL 
[ 60. 130.  60.  60.  60.], (VFL 
[ 60. 120.  70.  60.  60.], (VFL 
[130.  60.  60.  70.  60.], (VFL 
[70. 60. 60. 60. 60.], (VFL 
[70. 60. 60. 60. 70.], (VFL 
[60. 70. 60. 70. 60.], (VFL 
[

[230. 230. 210. 220. 220.], (N 
[240. 220. 230. 230. 220.], (N 
[220. 230. 220. 220. 220.], (N 
[240. 220. 220. 220. 230.], (N 
[230. 220. 220. 230. 230.], (N 
[220. 240. 230. 200. 210.], (N 
[250. 210. 220. 240. 220.], (N 
[230. 220. 220. 220. 220.], (N 
[240. 220. 220. 240. 220.], (N 
[220. 230. 230. 220. 230.], (N 
[240. 210. 230. 240. 210.], (N 
[220. 240. 220. 220. 230.], (N 
[230. 220. 230. 240. 220.], (N 
[230. 220. 240. 220. 220.], (N 
[230. 230. 230. 220. 230.], (N 
[230. 230. 220. 240. 220.], (N 
[230. 240. 220. 220. 240.], (N 
[240. 220. 230. 240. 210.], (N 
[240. 240. 220. 220. 240.], (N 
[220. 230. 240. 220. 220.], (N 
[200. 260. 240. 230. 240.], (N 
[230. 230. 220. 240. 220.], (N 
[230. 240. 220. 230. 240.], (N 
[230. 220. 240. 230. 220.], (N 
[240. 230. 230. 220. 230.], (N 
[230. 230. 230. 230. 220.], (N 
[240. 230. 220. 230. 230.], (N 
[230. 220. 240. 220. 230.], (N 
[230. 240. 220. 220. 240.], (N 
[230. 220. 240. 220. 220.], (N 
[220. 220. 240. 230. 230.], (N 
[230. 23

[50. 80. 60. 60. 60.], (VT 
[60. 60. 60. 60. 60.], (VT 
[60. 60. 60. 60. 60.], (VT 
[70. 60. 60. 60. 60.], (VT 
[60. 60. 60. 70. 50.], (VT 
[70. 60. 60. 60. 60.], (VT 
[60. 60. 60. 60. 60.], (VT 
[60. 60. 60. 70. 60.], (VT 
[60. 60. 60. 60. 60.], (VT 
[70. 50. 60. 70. 60.], (VT 
[60. 60. 60. 60. 60.], (VT 
[70. 60. 60. 60. 60.], (VT 
[60. 60. 60. 60. 60.], (VT 
[70. 60. 60. 60. 60.], (VT 
[60. 60. 60. 60. 70.], (VT 
[60. 60. 60. 60. 70.], (VT 
[ 60.  60.  60.  60. 120.], (VT 
[180.  70.  60.  60.  60.], (VT 
[70. 60. 60. 60. 60.], (VT 
[60. 60. 60. 60. 70.], (VT 
[60. 60. 60. 70. 60.], (VT 
[60. 60. 70. 60. 60.], (VT 
[ 60. 120.  70.  70.  70.], (VT 
[70. 60. 60. 60. 60.], (VT 
[70. 90. 90. 60. 60.], (VT 
[60. 60. 60. 50. 70.], (VT 
[ 80.  60. 100.  80.  70.], (VT 
[ 30. 100.  40.  60. 170.], (VT 
[ 90.  60.  60. 180.  80.], (VT 
[ 30.  60. 120.  60.  60.], (VT 
[ 50. 120.  60.  50. 140.], (VT 
[ 60.  50.  50. 110.  50.], (VT 
[70. 30. 60. 90. 60.], (VT 
[ 20.  80.  40.  50. 120.], (VT

[140.  50. 100.  90.  90.], (VT 
[140.  50. 140.  40. 130.], (VT 
[ 80.  50. 100.  50.  60.], (VT 
[100.  90. 100.  70. 100.], (VT 
[ 40. 100. 170. 100.  50.], (VT 
[ 40.  90. 140.  40. 130.], (VT 
[ 40. 100. 100.  90.  90.], (VT 
[ 90.  90.  40. 110.  90.], (VT 
[ 40. 140. 130. 100.  90.], (VT 
[ 60. 140. 130.  60. 170.], (VT 
[100.  50.  90.  50. 120.], (VT 
[130. 190.  90.  30. 180.], (VT 
[100. 140. 140. 320. 160.], (VT 
[130. 170. 230.  50. 140.], (VT 
[140. 160. 120.  90. 140.], (VT 
[ 80. 180.  60. 140.  70.], (VT 
[120. 170. 160. 140.  70.], (VT 
[ 60. 140.  90. 110. 170.], (VT 
[180.  50.  40. 130.  40.], (VT 
[ 90.  80.  50. 110. 100.], (VT 
[140.  40. 100.  40.  50.], (VF 
[140.  50. 130.  90. 140.], (VF 
[ 50.  50. 100.  60. 100.], (VF 
[90. 90. 90. 80. 50.], (VF 
[90. 90. 50. 90. 80.], (VF 
[ 40. 140.  60.  50. 140.], (VF 
[ 50. 180.  50.  90.  40.], (VF 
[190.  50. 160. 100. 150.], (VF 
[ 40. 140.  70.  90. 130.], (VF 
[130. 120.  50. 130. 100.], (VF 
[ 80. 190.  50.  50.

...

[344. 340. 330. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 330. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[330. 340. 340. 330. 350.], HGEA-N-NOD-PM-SBR-VER
[330. 340. 340. 340. 330.], HGEA-N-NOD-PM-SBR-VER
[340. 340. 330. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 330. 330. 350. 330.], HGEA-N-NOD-PM-SBR-VER
[190. 330. 350. 340. 330.], HGEA-N-NOD-PM-SBR-VER
[350. 330. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[330. 340. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[330. 340. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[330. 340. 340. 340. 330.], HGEA-N-NOD-PM-SBR-VER
[340. 340. 340. 330. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 330. 340. 340. 330.], HGEA-N-NOD-PM-SBR-VER
[190. 330. 350. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 340. 330. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 340. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 330. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 340. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[340. 330. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER
[330. 340. 340. 340. 340.], HGEA-N-NOD-PM-SBR-VER


[381. 195. 103.  83. 263.], HGEA-N-NOD-PM-SBR-VER
[182.  96.  80.  88. 284.], HGEA-N-NOD-PM-SBR-VER
[270. 100.  91. 394. 194.], HGEA-N-NOD-PM-SBR-VER
[342. 264.  79.  83.  88.], HGEA-N-NOD-PM-SBR-VER
[ 93.  97. 314. 235.  82.], HGEA-N-NOD-PM-SBR-VER
[108. 189. 165. 384. 156.], HGEA-N-NOD-PM-SBR-VER
[ 92.  81. 368. 187.  98.], HGEA-N-NOD-PM-SBR-VER
[ 79. 277. 169. 130. 372.], HGEA-N-NOD-PM-SBR-VER
[184.  93.  81.  86. 275.], HGEA-N-NOD-PM-SBR-VER
[184.  95. 349. 182. 105.], HGEA-N-NOD-PM-SBR-VER
[ 80.  90. 341. 172. 167.], HGEA-N-NOD-PM-SBR-VER
[ 78. 212. 162. 361. 188.], HGEA-N-NOD-PM-SBR-VER
[ 92. 206. 318. 217. 354.], HGEA-N-NOD-PM-SBR-VER
[346. 341. 339. 333. 335.], HGEA-N-NOD-PM-SBR-VER
[324. 333. 330. 281. 327.], HGEA-N-NOD-PM-SBR-VER
[335. 184. 186. 327. 346.], HGEA-N-NOD-PM-SBR-VER
[336. 343. 333. 337. 332.], HGEA-N-NOD-PM-SBR-VER
[303. 334. 329. 337. 326.], HGEA-N-NOD-PM-SBR-VER
[313. 301. 215. 322. 336.], HGEA-N-NOD-PM-SBR-VER
[327. 333. 255. 331. 290.], HGEA-N-NOD-PM-SBR-VER


[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 220.  60.  60.], B-VT-VF-VFL-ASYS
[100. 110. 120. 110. 110.], B-VT-VF-VFL-ASYS
[110. 100. 110. 230. 110.], B-VT-VF-VFL-ASYS
[110. 100. 110. 120. 110.], B-VT-VF-VFL-ASYS
[100. 110. 110. 120. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 160.], B-VT-VF-VFL-ASYS
[ 60. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[100. 120. 100. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 100. 120. 110. 100.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 100. 110. 110. 120.], B-VT-VF-VFL-ASYS
[100. 110. 110. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110. 100. 110. 110.], B-VT-VF-VFL-ASYS
[120. 110. 100. 110. 110.], B-VT-VF-VFL-ASYS
[110. 110.

[ 80. 230. 120.  60.  70.], B-VT-VF-VFL-ASYS
[150.  90. 130. 120.  60.], B-VT-VF-VFL-ASYS
[130. 190.  40. 140.  70.], B-VT-VF-VFL-ASYS
[ 60. 180.  40. 150. 120.], B-VT-VF-VFL-ASYS
[260.  90. 130. 130. 150.], B-VT-VF-VFL-ASYS
[ 60.  60.  90.  90. 320.], B-VT-VF-VFL-ASYS
[120.  60.  70. 110.  70.], B-VT-VF-VFL-ASYS
[ 60. 120. 120.  70.  60.], B-VT-VF-VFL-ASYS
[ 60.  60. 120. 160.  90.], B-VT-VF-VFL-ASYS
[190. 110. 110. 970. 190.], B-VT-VF-VFL-ASYS
[120. 190.  80.  40. 120.], B-VT-VF-VFL-ASYS
[130.  60.  60. 130. 400.], B-VT-VF-VFL-ASYS
[ 40. 180. 120. 100. 200.], B-VT-VF-VFL-ASYS
[140.  70. 170.  70. 150.], B-VT-VF-VFL-ASYS
[ 90. 130.  40.  80.  70.], B-VT-VF-VFL-ASYS
[ 60.  60. 130.  90.  30.], B-VT-VF-VFL-ASYS
[130.  90.  30.  70.  60.], B-VT-VF-VFL-ASYS
[ 60.  60.  60. 130.  60.], B-VT-VF-VFL-ASYS
[70. 60. 60. 70. 60.], B-VT-VF-VFL-ASYS
[ 60.  70. 190. 250. 230.], B-VT-VF-VFL-ASYS
[30. 70. 60. 70. 60.], B-VT-VF-VFL-ASYS
[60. 70. 60. 70. 60.], B-VT-VF-VFL-ASYS
[60. 70. 60. 60. 70.], B-

[180.  70.  50. 110.  50.], B-VT-VF-VFL-ASYS
[ 90.  80. 100. 100. 100.], B-VT-VF-VFL-ASYS
[ 80. 100.  70. 140. 120.], B-VT-VF-VFL-ASYS
[ 80.  80.  50. 110. 120.], B-VT-VF-VFL-ASYS
[110.  90. 160. 130. 130.], B-VT-VF-VFL-ASYS
[ 70.  40. 150.  60. 130.], B-VT-VF-VFL-ASYS
[130. 170. 130. 140.  80.], B-VT-VF-VFL-ASYS
[ 40. 110. 110. 120.  70.], B-VT-VF-VFL-ASYS
[ 80. 100. 100. 120. 100.], B-VT-VF-VFL-ASYS
[ 40. 100.  70.  70. 120.], B-VT-VF-VFL-ASYS
[140. 110. 110.  30. 210.], B-VT-VF-VFL-ASYS
[ 80. 170.  80. 120. 130.], B-VT-VF-VFL-ASYS
[ 90. 150.  50.  60.  50.], B-VT-VF-VFL-ASYS
[170.  80. 110.  80. 150.], B-VT-VF-VFL-ASYS
[150.  40. 170.  80. 130.], B-VT-VF-VFL-ASYS
[ 50.  80. 210.  60. 100.], B-VT-VF-VFL-ASYS
[170. 100. 100.  70. 120.], B-VT-VF-VFL-ASYS
[160.  60.  70. 110. 140.], B-VT-VF-VFL-ASYS
[100. 150. 130. 130.  90.], B-VT-VF-VFL-ASYS
[110.  60.  80. 110. 150.], B-VT-VF-VFL-ASYS
[ 90. 130.  80. 110.  90.], B-VT-VF-VFL-ASYS
[ 40.  70. 120.  60. 110.], B-VT-VF-VFL-ASYS
[130.  80.